# <strong>3b: Apex Only Notebook |</strong>


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks

In [28]:
# Load the data
df_5003_phase = pd.read_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/ELAN_FILES/5003_I_phase_tier.csv')
df_5003_processed = pd.read_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/5003_I_board_one_right_wrist_processed_data.csv')
output = 'C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/5003_I_board_one_apexes.csv'

The original R code for the unwrapper was a function so I reversed engineered it as a function

In [29]:
def align_phase(df_phase, df_processed):
    aligned_phase = []
    gesture_id = int(1)  # initialize gesture_id to 1
    gesture_id_list = []  # create an empty list to store gesture_id values
    phase_iter = iter(df_phase.itertuples())
    current_phase_row = next(phase_iter)
    
    for row in df_processed.itertuples():
        time_ms = row.time_ms
        while current_phase_row is not None and time_ms > current_phase_row._2:  # _2 corresponds to 'End Time'
            try:
                current_phase_row = next(phase_iter)
                gesture_id += int(1)  # increment gesture_id for each new annotation instance
            except StopIteration:
                current_phase_row = None
                
        if current_phase_row is not None and time_ms >= current_phase_row._1:  # _1 corresponds to 'Begin Time'
            aligned_phase.append(current_phase_row._3)  # append Gesture Phase value
            gesture_id_list.append(gesture_id)  # append gesture_id value to the list
        else:
            aligned_phase.append(None)  # append None for Gesture Phase value
            gesture_id_list.append(None)  # append None for gesture_id value
    
    df_processed['gesture_id'] = gesture_id_list  # add gesture_id list as a new column to df_processed
    return aligned_phase

In [30]:
df_5003_processed.head()

,right_wrist_x,right_wrist_y,speed_unsmooth,speed_smooth,time_ms
0,0.310565,0.599996,NaN,NaN,0.0
1,0.310135,0.600007,NaN,NaN,10.0
2,0.309705,0.600017,NaN,NaN,20.0
3,0.309275,0.600028,0.038672,NaN,30.0
4,0.308915,0.599960,0.036750,NaN,40.0


cleaned the data since I haven't integrated it earlier in the workflow 

In [31]:
# Add and align the 'phase' column
df_5003_processed['phase'] = align_phase(df_5003_phase, df_5003_processed)

# Keep only certain columns
df_5003_processed = df_5003_processed.loc[:, ['time_ms', 'speed_smooth', 'gesture_id', 'phase']]

more cleaning and the algorithm for the first peak detected within the stroke boundary and then marking it as such

In [32]:
# Filter out rows where 'speed_smooth' and 'phase' are not null
filtered_df = df_5003_processed.dropna(subset=['speed_smooth', 'phase'])

# Initialize a new empty DataFrame to hold the more refined annotations
more_refined_apexes_df = pd.DataFrame(columns=['time_ms', 'speed_smooth', 'gesture_id', 'apex'])

# Loop through each unique gesture_id
for gesture_id in filtered_df['gesture_id'].dropna().unique():
    # Filter data for the current gesture_id
    gesture_data = filtered_df[filtered_df['gesture_id'] == gesture_id]
    
    # Further filter to get data points where phase is 'S' or 'Stroke'
    stroke_data = gesture_data[gesture_data['phase'].isin(['S', 'Stroke'])]
    
    # Continue to next iteration if no 'Stroke' phase exists for this gesture_id
    if len(stroke_data) == 0:
        continue

    # Dynamically set the large peak threshold for this stroke phase
    dynamic_large_peak_threshold = stroke_data['speed_smooth'].mean() + stroke_data['speed_smooth'].std()

    # Find peaks within this 'Stroke' phase above the dynamic threshold
    peaks, _ = find_peaks(stroke_data['speed_smooth'].values, height=dynamic_large_peak_threshold)
    
    # If a large peak is found, search for the minimum speed after it
    if len(peaks) > 0:
        for peak in peaks:
            subsequent_data = stroke_data['speed_smooth'].values[peak:]
            if len(subsequent_data) > 1:
                min_speed_index = np.argmin(subsequent_data) + peak
                # Map the min_speed_index to the original DataFrame's index
                original_index = stroke_data.index[min_speed_index]
                stroke_data.loc[original_index, 'apex'] = 'AX'
            
    # Append this stroke_data to more_refined_apexes_df
    more_refined_apexes_df = more_refined_apexes_df.append(stroke_data)

###################### I disabled the plotting code below because it was being annoying and so I was just subsetting it. It's too dense to see much anyway ######################

# # Sample the annotated data for plotting
# sampled_more_refined_annotated_df = more_refined_apexes_df.dropna(subset=['apex']).sample(frac=0.2, random_state=1)

# # Plotting with more refined annotations along with speed curve
# plt.figure(figsize=(15, 6))
# plt.plot(filtered_df['time_ms'], filtered_df['speed_smooth'], label='Speed Smooth Curve', alpha=0.7)

# # Plot the apexes on the speed curve
# plt.scatter(sampled_more_refined_annotated_df['time_ms'], sampled_more_refined_annotated_df['speed_smooth'], c='purple', label='Apexes', alpha=0.7, zorder=5)

# # Annotate sampled apexes
# for i, txt in enumerate(sampled_more_refined_annotated_df['apex']):
#     plt.annotate(txt, (sampled_more_refined_annotated_df['time_ms'].iloc[i], sampled_more_refined_annotated_df['speed_smooth'].iloc[i]), fontsize=8, alpha=0.7)

# plt.title('Speed Smooth Curve with More Refined Apex Annotations')
# plt.xlabel('Time (ms)')
# plt.ylabel('Speed (Smoothed)')
# plt.legend()
# plt.show()

more_refined_apexes_df.to_csv(output, index=False)


C:\Users\cosmo\AppData\Local\Temp\ipykernel_24812\3633481.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  more_refined_apexes_df = more_refined_apexes_df.append(stroke_data)
C:\Users\cosmo\AppData\Local\Temp\ipykernel_24812\3633481.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  more_refined_apexes_df = more_refined_apexes_df.append(stroke_data)
C:\Users\cosmo\AppData\Local\Temp\ipykernel_24812\3633481.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  more_refined_apexes_df = more_refined_apexes_df.append(stroke_data)
C:\Users\cosmo\AppData\Local\Temp\ipykernel_24812\3633481.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead